# EX9.프로젝트 - Movielens 영화 추천하기

## Movielens 데이터
-----------------------
* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. (MovieLens 1M Dataset 사용을 권장) 

* 별점 데이터는 대표적인 explicit 데이터지만 implicit 데이터로 간주하고 테스트해볼 수 있다.   

* 별점을 **시청횟수**로 해석해서 생각하겠음  

* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠음  

## - 목차 -   

### 1) 데이터 준비와 전처리 
### 2) 분석해보기 
### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가하기 
### 4) CSR matrix 만들기 
### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련하기 
### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악 
### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 보기 
### 8) 내가 가장 좋아할 만한 영화들을 추천받아 보기 
### 9) 추천시스템에 기여한 정도 확인하기

## 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

# Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있음
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


* id와 title을 확인하는 함수 생성

In [39]:
def id_to_title(id):
    return movies['title'][movies['movie_id']==id].values[0]

In [42]:
id_to_title(10)

'GoldenEye (1995)'

In [40]:
def title_to_id(title):
    return movies['movie_id'][movies['title']==title].values[0].item()

In [43]:
title_to_id('GoldenEye (1995)')

10

## 2) 분석해보기
-----------------------
* ratings에 있는 유니크한 영화 개수     

* rating에 있는 유니크한 사용자 수       

* 가장 인기 있는 영화 30개(인기순)      

In [7]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['user_id'].nunique()

6039

In [9]:
# rating에 있는 유니크한 사용자 수
ratings['movie_id'].nunique()

3628

In [16]:
# 인기 많은 영화
movie_data = pd.merge(ratings, movies)
movies_count = movie_data.groupby('title')['user_id'].count() # 영화별 grouping, 사용자 수를 카운트
movies_count.sort_values(ascending=False).head(40)              # 상위 40편

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [14]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['timestamp'].median()
user_median.describe()

count    6.039000e+03
mean     9.705019e+08
std      8.916703e+06
min      9.567058e+08
25%      9.650976e+08
50%      9.720067e+08
75%      9.748566e+08
max      1.045440e+09
Name: timestamp, dtype: float64

In [15]:
ratings.describe()

,user_id,movie_id,count,timestamp
count,836478.000000,836478.000000,836478.000000,8.364780e+05
mean,3033.120626,1849.099114,3.958293,9.721628e+08
std,1729.255651,1091.870094,0.762280,1.206216e+07
min,1.000000,1.000000,3.000000,9.567039e+08
25%,1531.000000,1029.000000,3.000000,9.652795e+08
50%,3080.000000,1747.000000,4.000000,9.728388e+08
75%,4485.000000,2763.000000,5.000000,9.752064e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가하기

In [17]:
# 내가 좋아하는 영화 5편 추가

my_fav = ['Toy Story (1995)' , 'Jumanji (1995)' ,'E.T. the Extra-Terrestrial (1982)' ,'Jurassic Park (1993)' ,'Wizard of Oz, The (1939)']


# user_id를 'kwak'으로 정하고, 좋아하는 영화이니 모두 5점을 부여함
my_movies = pd.DataFrame({'user_id': ['kwak']*5, 'movie_id': my_fav, 'count':[5]*5, 'timestamp':[972162800]*5})

if not ratings.isin({'user_id':['kwak']})['user_id'].any():  # user_id에 동일 id가 있는지 확인 후 추가
    ratings = ratings.append(my_movies)                            

ratings.tail(10)       # append는 맨 뒤에 붙인다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
0,kwak,Toy Story (1995),5,972162800
1,kwak,Jumanji (1995),5,972162800
2,kwak,E.T. the Extra-Terrestrial (1982),5,972162800
3,kwak,Jurassic Park (1993),5,972162800
4,kwak,"Wizard of Oz, The (1939)",5,972162800


In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [19]:
user_to_idx['kwak']

6039

In [20]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna() # .dropna() : 결측치(누락데이터) 제거
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 주기
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 주기
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie id column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760
1,0,1,3,978302109
2,0,2,3,978301968
3,0,3,4,978300275
4,0,4,5,978824291
...,...,...,...,...
0,6039,3628,5,972162800
1,6039,3629,5,972162800
2,6039,3630,5,972162800
3,6039,3631,5,972162800


## 4) CSR matrix 만들기

CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도

Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조

In [21]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id'])), shape=(num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련하기

In [22]:
from implicit.als import AlternatingLeastSquares # 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
# Implicit AlternatingLeastSquares 모델 선언
# factors,iterations를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 함
als_model = AlternatingLeastSquares(factors=3000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해주기)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기

In [26]:
kwak, toy_story = user_to_idx['kwak'], movie_to_idx['Toy Story (1995)']
kwak_vector, toy_story_vector = als_model.user_factors[kwak], als_model.item_factors[toy_story]

In [27]:
kwak_vector

array([-0.1238621 , -0.20149468, -0.38748214, ..., -0.23125648,
        0.23628765,  0.44795123], dtype=float32)

In [28]:
toy_story_vector

array([0.00631138, 0.00656622, 0.00351129, ..., 0.00113653, 0.00947476,
       0.00654186], dtype=float32)

* 두 벡터를 곱해서 1에 가까운 수가 나올 수록 학습이 잘 된 것

In [29]:
np.dot(kwak_vector, toy_story_vector)

0.9214159

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 보기

In [30]:
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_artist = als_model.similar_items(movie_id, N=15)
similar_artist

[(3628, 1.0000001),
 (3630, 0.8963051),
 (3629, 0.89235836),
 (3631, 0.88348645),
 (3632, 0.878075),
 (3409, 0.86967057),
 (3456, 0.8692922),
 (3587, 0.8691476),
 (3429, 0.86906457),
 (3561, 0.86796737),
 (3577, 0.8669072),
 (3609, 0.86550796),
 (3343, 0.86513335),
 (3567, 0.8650099),
 (3246, 0.86495864)]

* 장르 확인해보기

In [45]:
# (movie_id, 유사도)
# movie_id를 tilte로 변환
for i in [id_to_title(i[0]) for i in similar_artist]:
    print(movies[movies['title']==i]['genre'].unique())
[id_to_title(i[0]) for i in similar_artist]

['Action|Drama|War']
['Horror']
['Comedy']
['Comedy']
['Comedy']
['Drama|Thriller']
['Drama']
['Horror']
['Animation|Comedy']
['Drama']
['Drama']
['Documentary']
['Comedy|Drama|Romance']
['Comedy']
['Drama']


['Flying Tigers (1942)',
 "House of Exorcism, The (La Casa dell'esorcismo) (1974)",
 'Gold Rush, The (1925)',
 "It's in the Water (1998)",
 'Monsieur Verdoux (1947)',
 'Final Destination (2000)',
 'Color of Paradise, The (Rang-e Khoda) (1999)',
 'Inferno (1980)',
 'Creature Comforts (1990)',
 "Stacy's Knights (1982)",
 'Two Moon Juction (1988)',
 'Regret to Inform (1998)',
 'And God Created Woman (1988)',
 'Bossa Nova (1999)',
 'Malcolm X (1992)']

In [46]:
# similar_movie확인하는 함수
def get_similar_movie(movie_name: str):
    movie_name = movie_name
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [id_to_title(i[0]) for i in similar_movie]
    return similar_movie

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 보기

In [47]:
user = user_to_idx['kwak']
# recommend에서는 user*item CSR Matrix를 받음
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3260, 0.075295866),
 (3531, 0.07407002),
 (2215, 0.06186974),
 (3550, 0.06134496),
 (3038, 0.059443653),
 (3112, 0.058794107),
 (2736, 0.058478035),
 (2996, 0.05827696),
 (2653, 0.0577458),
 (3089, 0.05692556),
 (3477, 0.05640951),
 (3549, 0.05615197),
 (3203, 0.055640586),
 (3102, 0.05543867),
 (3461, 0.053905345),
 (3532, 0.051653236),
 (3201, 0.049594723),
 (2990, 0.049263272),
 (3409, 0.049019624),
 (2733, 0.0486589)]

In [48]:
[id_to_title(i[0]) for i in movie_recommended]

['Howards End (1992)',
 'All the Vermeers in New York (1990)',
 'Rich and Strange (1932)',
 'Hunger, The (1983)',
 'Face in the Crowd, A (1957)',
 "'Night Mother (1986)",
 'Brighton Beach Memoirs (1986)',
 'Music of the Heart (1999)',
 'Son of Dracula (1943)',
 'Bicycle Thief, The (Ladri di biciclette) (1948)',
 'Empire Records (1995)',
 'Guys and Dolls (1955)',
 'Dead Calm (1989)',
 'Jagged Edge (1985)',
 'Lord of the Flies (1963)',
 'Freedom for Us (À nous la liberté ) (1931)',
 'Five Easy Pieces (1970)',
 'Licence to Kill (1989)',
 'Final Destination (2000)',
 'Vibes (1988)']

In [50]:
# 장르 유사도
for i in [id_to_title(i[0]) for i in movie_recommended]:
    print(movies[movies['title']==i]['genre'].unique())

['Drama']
['Comedy|Drama|Romance']
['Comedy|Romance']
['Horror']
['Drama']
['Drama']
['Comedy']
['Drama']
['Horror']
['Drama']
['Comedy|Drama']
['Musical']
['Thriller']
['Thriller']
['Adventure|Drama|Thriller']
['Comedy']
['Drama']
['Action']
['Drama|Thriller']
['Adventure|Comedy']


호러를 제외하고는 대부분 좋아하는 장르가 나옴

## 9) 추천시스템에 기여한 정도 확인하기
* AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [52]:
Toy_Story = movie_to_idx['Toy Story (1995)']
# 이 explain method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됨)를 반환
explain = als_model.explain(user, csr_data, itemid=Toy_Story)

In [53]:
explain

(0.9170625569833621,
 [(3628, 0.6530092711188937),
  (3630, 0.08824766124000716),
  (3629, 0.0726959603680134),
  (3631, 0.05655192770930732),
  (3632, 0.04655773654714063)],
 (array([[ 0.49694544,  0.20120958,  0.20746468, ...,  0.2152635 ,
           0.19750612,  0.20150556],
         [ 0.09999019,  0.43546778,  0.11832413, ...,  0.11032163,
           0.10172915,  0.1181941 ],
         [ 0.10309863,  0.09327023,  0.42015361, ...,  0.08809781,
           0.08876102,  0.09174   ],
         ...,
         [ 0.10697422,  0.0913546 ,  0.0947279 , ...,  0.28312013,
           0.00550721,  0.00828479],
         [ 0.09814977,  0.08403989,  0.09030582, ...,  0.09493105,
           0.28388738, -0.00149628],
         [ 0.10013727,  0.09201457,  0.09433539, ...,  0.09706461,
           0.08971476,  0.28578353]]),
  False))

In [55]:
[(id_to_title(i[0]), i[1]) for i in explain[1]]

[('Flying Tigers (1942)', 0.6530092711188937),
 ("House of Exorcism, The (La Casa dell'esorcismo) (1974)",
  0.08824766124000716),
 ('Gold Rush, The (1925)', 0.0726959603680134),
 ("It's in the Water (1998)", 0.05655192770930732),
 ('Monsieur Verdoux (1947)', 0.04655773654714063)]

* 'Flying Tigers (1942)'가 기여도가 가장 높게 나왔다. 장르는 액션인데 선호하는 영화중 장르가 액션영화인 'Jumanji'와 유사하고
* 'Monsieur Verdoux (1947)'는 기여도가 가장 낮게 나왔는데 장르는 코미디이다. 코미디는 선호하는 영화 장르에 없었다.

# 🌟 루브릭 평가

|평가문항|상세문항|
|:------|:---|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

#### 1. CSR matrix가 정상적으로 만들어졌다.    
   -> CSR matrix는 잘 만들어진 것 같다.  

#### 2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
   ->  대부분 선호한 영화와 비슷한 영화가 추천됨 

#### 3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.	
   -> 유사도는 의미있게 측정이 된 것 같지는 않지만(호러,스릴러가 나옴..) 기여도는 유사도에 비해 의미있게 측정된 것 같다.

# 🌟 회고

요즘 인터넷쇼핑을 하거나 유튜브를 보는 등 일상에서 추천시스템을 활용한 사례를 흔히 볼 수 있는데 평소에 '어떤 원리로 추천을 해주는 걸까' 하고 궁금했었는데 
그 주제로 프로젝트를 진행하게 되어서 재미있었다.  \
Matrix Factorization(MF)와 CSR matrix에 대한 내용도 알 수 있었고, 전체적으로 코드를 다시한번 자세하게 보고 공부하면서 영화추천이 아니라 음악추천도 한번 해보고 싶다는 생각이 들었다.

## ⭐️참고
[miinkang님 코드](https://github.com/miinkang/AI_Project_AIFFEL/blob/main/%5BE-08%5DMovielens_recommendation.ipynb)